Follow https://github.com/robmarkcole/HASS-data-detective/blob/master/usage/Usage%20of%20detective.ipynb

Load Olis hologram data. Note that the data is not valid json as Oli has used single rather than double quotations. **JSON requires double quotes for its strings.**

In [1]:
# To import detective from relative folder# To im 
import json
import os
import sys
print(sys.version)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import detective.core as detective
import detective.helpers as helpers
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # Suppress chained_assignment warning, default='warn'
import seaborn as sns
from sqlalchemy import text
import matplotlib.pyplot as plt
%matplotlib inline

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 12:04:33) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
db_path = '/Users/robincole/Documents/Data/Home-Assistant/Olis_db/home-assistant_v2.db'
DB_URL = 'sqlite:////' + db_path

In [3]:
DB_URL

'sqlite://///Users/robincole/Documents/Data/Home-Assistant/Olis_db/home-assistant_v2.db'

In [4]:
%%time
db = detective.HassDatabase(DB_URL) # To init without fetching entities fetch_entities=False

Successfully connected to database
There are 49 entities with data
CPU times: user 96.5 ms, sys: 127 ms, total: 224 ms
Wall time: 711 ms


In [5]:
db.domains

['light',
 'sensor',
 'binary_sensor',
 'group',
 'hello',
 'automation',
 'device_tracker',
 'zone',
 'input_select',
 'hello_world',
 'switch',
 'input_number',
 'sun',
 'persistent_notification',
 'updater',
 'camera']

In [6]:
db.entities['sensor']

['sensor.cpu_temperature',
 'sensor.dark_sky_temperature',
 'sensor.dark_sky_nearest_storm_distance',
 'sensor.dark_sky_summary',
 'sensor.dark_sky_icon',
 'sensor.aqm_message_time',
 'sensor.aqm_pressure',
 'sensor.hologram_155807',
 'sensor.aqm_temperature',
 'sensor.aqm_pm10',
 'sensor.aqm_pm25',
 'sensor.aqm_pm1',
 'sensor.aqm_humidity',
 'sensor.deluge_up_speed',
 'sensor.deluge_down_speed',
 'sensor.weather_symbol',
 'sensor.yr_symbol',
 'sensor.ssl_cert_expiry',
 'sensor.deluge_status']

In [7]:
query = text(
    """
    SELECT state, last_changed
    FROM states
    WHERE entity_id in ('sensor.hologram_155807') 
    AND NOT state='unknown'
    """
    )

In [8]:
response = db.perform_query(query)

In [9]:
hologram_155807 = response.fetchall()

In [10]:
reading_1 = hologram_155807[0]
reading_1_items = reading_1.items()

In [11]:
reading_1_data = reading_1_items[0][1]
reading_1_data

"{'Temperature': 21.9092973805, 'PM25': 3, 'Humidity': 0, 'Pressure': 100650.718305, 'PM1': 2, 'PM10': 3}"

In [12]:
type(reading_1_data)

str

We want to parse this string into a dict. Note that it is not formatted correctly and we will get an error because of formatting, see https://stackoverflow.com/questions/39491420/python-jsonexpecting-property-name-enclosed-in-double-quotes

Lets hand code the valid json

In [13]:
valid_json = """{"Temperature": 21.9092973805, "PM25": 3, "Humidity": 0, "Pressure": 100650.718305, "PM1": 2, "PM10": 3}"""
valid_json

'{"Temperature": 21.9092973805, "PM25": 3, "Humidity": 0, "Pressure": 100650.718305, "PM1": 2, "PM10": 3}'

In [14]:
reading_1_data_dict = json.loads(valid_json)   
reading_1_data_dict

{'Temperature': 21.9092973805,
 'PM25': 3,
 'Humidity': 0,
 'Pressure': 100650.718305,
 'PM1': 2,
 'PM10': 3}

In [15]:
reading_1_data_dict['Temperature']

21.9092973805